In [1]:
from collections import Counter
import math
import numpy as np

import pandas as pd
#from qpsolvers import solve_qp


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

src_path = '/content/drive/My\ Drive/CMSC422/HW1/20_newsgroups.tar.gz'
extract_path = '/content/20_newsgroups'

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

!tar -xzf {src_path} -C {extract_path}


Mounted at /content/drive


In [3]:
dataset_path = '/content/20_newsgroups/20_newsgroups'

newsgroup_classes = os.listdir(dataset_path)
print("Newsgroup Categories:", newsgroup_classes)

sample_class_path = os.path.join(dataset_path, newsgroup_classes[0])
sample_files = os.listdir(sample_class_path)
print("\nSample files in category '{}':".format(newsgroup_classes[0]), sample_files[:5])

Newsgroup Categories: ['rec.sport.baseball', 'sci.med', 'sci.electronics', 'comp.os.ms-windows.misc', 'talk.politics.mideast', 'talk.politics.misc', 'rec.autos', 'talk.politics.guns', 'comp.sys.mac.hardware', 'rec.sport.hockey', 'alt.atheism', 'talk.religion.misc', 'soc.religion.christian', 'comp.windows.x', 'comp.sys.ibm.pc.hardware', 'sci.crypt', 'comp.graphics', 'misc.forsale', 'sci.space', 'rec.motorcycles']

Sample files in category 'rec.sport.baseball': ['105087', '104896', '104789', '104775', '105121']


In [4]:
# Preprocessing
import os
import string
from collections import Counter

def process_all_files(dataset_path):

    all_words = []
    cleaned_data = {}

    for newsgroup in os.listdir(dataset_path):
        newsgroup_path = os.path.join(dataset_path, newsgroup)
        if os.path.isdir(newsgroup_path):
            cleaned_data[newsgroup] = []

            for filename in os.listdir(newsgroup_path):
                file_path = os.path.join(newsgroup_path, filename)

                with open(file_path, 'r', encoding='latin1') as file:
                    lines = file.readlines()

                # Remove the first four lines
                content = ''.join(lines[4:])

                # Convert to lowercase
                content = content.lower()

                # Remove punctuation
                content = content.translate(str.maketrans('', '', string.punctuation))

                # Split into words
                words = content.split()

                # Collect words for stop word calculation
                all_words.extend(words)


                cleaned_data[newsgroup].append((words, file_path))

    # Finding the top 200 words as stop words
    word_counter = Counter(all_words)
    top_300_stop_words = set([word for word, _ in word_counter.most_common(300)])

    final_cleaned_data = {}
    for newsgroup, files in cleaned_data.items():
        final_cleaned_data[newsgroup] = []

        for words, file_path in files:
            # Remove stop words, short words, and email addresses
            words_filtered = [
                word for word in words
                if word not in top_300_stop_words and
                len(word) > 2 and
                '@' not in word
            ]

            # Join filtered words
            cleaned_content = ' '.join(words_filtered)

            final_cleaned_data[newsgroup].append((cleaned_content, file_path))

    return final_cleaned_data




In [5]:
cleaned_dataset = process_all_files(dataset_path)

# Display a cleaned file
example_class = os.listdir(dataset_path)[0]
cleaned_content, file_path = cleaned_dataset[example_class][0]

print(f"\nSample cleaned email from '{example_class}':\n")
print(f"File Path: {file_path}")
print(f"Content: {cleaned_content[:500]}")


Sample cleaned email from 'rec.sport.baseball':

File Path: /content/20_newsgroups/20_newsgroups/rec.sport.baseball/105087
Content: 13405newsdukeedu 204747 newsnewsdukeedu duke durham keynesecondukeedu term stopper generally refer pitcher counted pitch strong keep team losing streak braves plenty pitchers fit description although expect smoltz glavine mantle braves lack offensive stopper somebody bring hitting slump theres braves rid pure hitter lonnie smith terry pendleton current roster ever shown cursory ability hit worries ron gant slowed step scary slow ron gant econdukeedu flsecondukeedu flsecondukeedu flseconduke corr


In [6]:
def prepare_data(train_dict):
    documents = []
    labels = []
    label_mapping = {label: idx for idx, label in enumerate(train_dict.keys())}

    for label, docs in train_dict.items():
        for content, _ in docs:
            documents.append(content)
            labels.append(label_mapping[label])

    return np.array(documents), np.array(labels), label_mapping


In [7]:
def compute_tfidf_limited(documents, top_k=500):
    N = len(documents)  # Total number of documents
    term_frequencies = []  # To store term frequencies for each document
    document_frequencies = Counter()  # To count document frequencies for each term

    # Calculate term frequencies and document frequencies
    for document in documents:
        tf = Counter(document.split())
        term_frequencies.append(tf)
        unique_terms = set(tf.keys())
        for term in unique_terms:
            document_frequencies[term] += 1

    # Get the top-k terms (features) by document frequency
    top_k_terms = [term for term, _ in document_frequencies.most_common(top_k)]

    # Calculate TF-IDF using the formula for top-k terms only
    tfidf_matrix = []
    for tf in term_frequencies:
        doc_tfidf = {}
        for term in top_k_terms:
            if term in tf:
                tf_value = math.log10(1 + tf[term])
                idf_value = math.log10(N / document_frequencies[term])
                tfidf = tf_value * idf_value
                doc_tfidf[term] = tfidf
            else:
                doc_tfidf[term] = 0
        tfidf_matrix.append(doc_tfidf)

    return tfidf_matrix, top_k_terms




In [9]:
!pip install qpsolvers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 3.4 MB/s eta 0:00:00


In [10]:
from qpsolvers import solve_qp
def linear_svm_train(X, y, C=100):

    N, D = X.shape

    P = np.zeros((D + N + 1, D + N + 1))
    P[:D, :D] = np.eye(D)


    q = np.hstack([np.zeros(D + 1), C * np.ones(N)])
    G = np.zeros((2 * N, D + N + 1))
    h = np.zeros(2 * N)

    for i in range(N):
        G[i, :D] = -y[i] * X[i]
        G[i, D] = -y[i]
        G[i, D + 1 + i] = -1
        h[i] = -1

        G[N + i, D + 1 + i] = -1



    solution = solve_qp(P, q, G, h, solver="cvxopt")

    w = solution[:D]
    b = solution[D]

    return w, b


In [11]:
def train_one_vs_all(X, y, C=100):

    classes = np.unique(y)
    classifiers = {}

    for cls in classes:
        print(f"Training classifier for class {cls}...")
        binary_labels = np.where(y == cls, 1, -1)
        w, b = linear_svm_train(X, binary_labels, C=C)
        classifiers[cls] = (w, b)

    return classifiers


In [12]:
def predict_one_vs_all(classifiers, X):

    scores = {}

    for cls, (w, b) in classifiers.items():
        scores[cls] = X @ w + b

    return np.array([max(scores, key=lambda cls: scores[cls][i]) for i in range(X.shape[0])])




In [53]:
#Spliting the datasets


train ={}
test = {}

for category, documents in cleaned_dataset.items():
  train[category] = documents[:500]
  test[category] = documents[500:]


example_class = list(cleaned_dataset.keys())[0]
print(f"Number of documents in '{example_class}' - Training: {len(train[example_class])}, Testing: {len(test[example_class])}")

Number of documents in 'rec.sport.baseball' - Training: 500, Testing: 500


In [14]:
X_train_documents, y_train_labels, label_mapping = prepare_data(train)
X_train_tfidf, top_500_terms = compute_tfidf_limited(X_train_documents, top_k=500)

train_tfidf_df = pd.DataFrame([{term: tfidf.get(term, 0) for term in top_500_terms} for tfidf in X_train_tfidf])

X_train_tfidf = train_tfidf_df.to_numpy()



classifiers = train_one_vs_all(X_train_tfidf, y_train_labels, C=100)



Training classifier for class 0...
Training classifier for class 1...
Training classifier for class 2...
Training classifier for class 3...
Training classifier for class 4...
Training classifier for class 5...
Training classifier for class 6...
Training classifier for class 7...
Training classifier for class 8...
Training classifier for class 9...
Training classifier for class 10...
Training classifier for class 11...
Training classifier for class 12...
Training classifier for class 13...
Training classifier for class 14...
Training classifier for class 15...
Training classifier for class 16...
Training classifier for class 17...
Training classifier for class 18...
Training classifier for class 19...


In [43]:
X_test_documents, y_test_labels, label_mapping = prepare_data(test)
X_test_tfidf, _ = compute_tfidf_limited(X_test_documents, top_k=500)

test_tfidf_df = pd.DataFrame([{term: tfidf.get(term, 0) for term in top_500_terms} for tfidf in X_test_tfidf])

X_test_tfidf = test_tfidf_df.to_numpy()

# Predict using the trained classifiers
y_test_pred = predict_one_vs_all(classifiers, X_test_tfidf)
print("Predicted labels:", y_test_pred)



Predicted labels: [14 19  9 ...  8 17 18]


In [51]:
accuracy = sum(y_test_pred == y_test_labels) / len(y_test_labels)
print("Accuracy:", accuracy)

Accuracy: 0.5568571428571428


In [37]:
def polynomial_kernel(X1, X2):
    return np.dot(X1, X2.T) ** 2

In [38]:
from cvxopt import matrix, solvers

def polynomial_soft_margin_svm(X, y_labels, C=100):
    n_samples = X.shape[0]
    q = matrix(-np.ones(n_samples))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))

    support_data = []

    unique_classes = np.unique(y_labels)
    for class_label in unique_classes:
        print(f"Training classifier for class {class_label}...")
        y_train = np.where(y_labels == class_label, 1, -1).astype(float)

        P = matrix(np.outer(y_train, y_train) * polynomial_kernel(X, X))
        A = matrix(y_train, (1, n_samples), 'd')
        b = matrix(0.0)

        solution = solvers.qp(P, q, G, h, A, b)

        alphas = np.ravel(solution['x'])
        support_indices = alphas > 1e-5
        alphas = alphas[support_indices]
        support_vectors = X[support_indices]
        support_labels = y_train[support_indices]

        support_data.append((alphas, support_vectors, support_labels))

    return support_data

In [35]:
def predict_polynomial_svm(X_test, support_data):
    predictions = []

    for test_point in X_test:
        class_scores = []

        for alphas, support_vectors, support_labels in support_data:
            K_test = polynomial_kernel(support_vectors, test_point.reshape(1, -1)).flatten()
            score = np.sum(alphas * support_labels * K_test)
            class_scores.append(score)

        predictions.append(np.argmax(class_scores))

    return predictions

In [52]:
# Training

support_data  = polynomial_soft_margin_svm(X_train_tfidf, y_train_labels, C=100)


Training classifier for class 0...


KeyboardInterrupt: 

In [50]:
#Testing
y_pred = predict_polynomial_svm(X_test_tfidf, support_data)


accuracy = np.mean(y_pred == y_test_labels)
print("Accuracy:", accuracy)

Accuracy: 0.7464285714285714
